In [ ]:
%mkdir yolov5_ws
%cd yolov5_ws
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

In [5]:
import time
import psutil  # For memory usage

def run_inference_on_video(model_path, input_video, output_video, conf_threshold=0.1):
    # Load YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

    # Set model confidence threshold
    model.conf = conf_threshold

    # Open the input video file
    video_capture = cv2.VideoCapture(input_video)
    if not video_capture.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    original_frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))

    # Define codec and create VideoWriter object
    video_writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), fps, (original_frame_width, original_frame_height))

    # Initialize metrics
    start_time = time.time()
    frame_count = 0
    total_inference_time = 0

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        frame_count += 1

        # Preprocess frame for YOLOv5 (resize to 640x640)
        resized_frame = preprocess_frame(frame)

        # Measure inference time
        inference_start = time.time()
        results = model(resized_frame)
        inference_end = time.time()

        # Update total inference time
        total_inference_time += (inference_end - inference_start)

        # Get bounding boxes and annotations
        boxes = results.xyxy[0].cpu().numpy()  # Get boxes as numpy array

        # Post-process boxes to original size
        boxes_rescaled = postprocess_boxes(boxes, (original_frame_height, original_frame_width), (640, 640))

        # Draw bounding boxes on the original frame
        for box in boxes_rescaled:
            x1, y1, x2, y2, conf, cls = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f'{model.names[cls]} {conf:.2f}'
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Perform lane detection
        frame_with_lanes = detect_lanes(frame)

        # Write the processed frame with both object detection and lane detection to the output video
        video_writer.write(frame_with_lanes)

        # Calculate and print status metrics every 10 frames
        if frame_count % 10 == 0:
            elapsed_time = time.time() - start_time
            avg_inference_time = total_inference_time / frame_count
            fps_current = frame_count / elapsed_time
            memory_usage = psutil.Process().memory_info().rss / (1024 ** 2)  # Memory usage in MB

            print(f"FPS: {fps_current:.2f}, Average Inference Time: {avg_inference_time:.4f} seconds, Memory Usage: {memory_usage:.2f} MB")

    # Release video objects
    video_capture.release()
    video_writer.release()

    print(f"Output video saved to: {output_video}")

# Define paths to model, input video, and output video
model_path = '/content/destilation_prunned_quantized_fixed.onnx'  # Path to the YOLOv5 model (PyTorch format)
input_video_path = '/content/test_video.mp4'  # Path to your input video
output_video_path = '/content/output_video.mp4'  # Path to save the output video

run_inference_on_video(model_path, input_video_path, output_video_path, conf_threshold=0.3)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-9-4 Python-3.10.12 torch-2.4.0+cu121 CPU

Loading /content/destilation_prunned_quantized_fixed.onnx for ONNX Runtime inference...
Adding AutoShape... 


FPS: 3.28, Average Inference Time: 0.2943 seconds, Memory Usage: 945.93 MB
FPS: 2.77, Average Inference Time: 0.3497 seconds, Memory Usage: 945.93 MB
FPS: 2.95, Average Inference Time: 0.3282 seconds, Memory Usage: 945.93 MB
FPS: 3.06, Average Inference Time: 0.3170 seconds, Memory Usage: 945.93 MB
FPS: 3.12, Average Inference Time: 0.3100 seconds, Memory Usage: 945.93 MB
FPS: 3.00, Average Inference Time: 0.3231 seconds, Memory Usage: 945.93 MB
FPS: 3.03, Average Inference Time: 0.3198 seconds, Memory Usage: 945.93 MB
FPS: 3.08, Average Inference Time: 0.3154 seconds, Memory Usage: 945.93 MB
FPS: 3.11, Average Inference Time: 0.3119 seconds, Memory Usage: 945.93 MB
FPS: 2.99, Average Inference Time: 0.3239 seconds, Memory Usage: 945.93 MB
FPS: 2.78, Average Inference Time: 0.3486 seconds, Memory Usage: 945.93 MB
FPS: 2.81, Average Inference Time: 0.3458 seconds, Memory Usage: 945.93 MB
FPS: 2.82, Average Inference Time: 0.3447 seconds, Memory Usage: 945.93 MB
FPS: 2.81, Average Infere